<a href="https://colab.research.google.com/github/vardansaini/Custom-T5/blob/main/CustomT5_pytorch_V3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import pandas as pd
import numpy as np
import torchtext
from collections import OrderedDict
from scipy import spatial

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
compound_data = pd.read_csv('/content/drive/MyDrive/all_embeddings_forML.csv')

In [6]:
voc = np.unique( compound_data[ ['c1', 'c2', 'cmp'] ].values.reshape(-1) )
# print(voc)

In [7]:
df = compound_data.copy()
# print(df.head())
df = df.sample(frac=1).reset_index(drop=True)
# print(df)

In [8]:
c1 = list(df['c1'])
c2 = list(df['c2'])
compounds = list(df['cmp'])
rows = len(compounds)

print(c1)
print(len(c1))
print(len(c2))
print(len(compounds))



['white', 'mocking', 'bread', 'snow', 'letter', 'song', 'hot', 'camp', 'feather', 'water', 'weather', 'dream', 'bull', 'bind', 'foot', 'weather', 'bulk', 'tin', 'end', 'moth', 'draw', 'joy', 'pan', 'light', 'flower', 'ground', 'shell', 'book', 'side', 'joy', 'troop', 'score', 'night', 'spokes', 'white', 'score', 'mocking', 'fortune', 'handle', 'play', 'melt', 'eye', 'block', 'hair', 'tea', 'date', 'wax', 'dug', 'search', 'book', 'cock', 'back', 'night', 'bell', 'gentle', 'bow', 'fire', 'rag', 'fresh', 'bull', 'green', 'waste', 'shelter', 'farm', 'fade', 'rock', 'rain', 'prim', 'moon', 'horse', 'after', 'bay', 'sledge', 'band', 'lens', 'door', 'cow', 'para', 'space', 'mast', 'camp', 'god', 'bob', 'monks', 'hay', 'pink', 'motor', 'shark', 'bed', 'grounds', 'heart', 'break', 'letter', 'bow', 'type', 'snow', 'apple', 'short', 'touch', 'flat', 'jump', 'over', 'hate', 'bone', 'fire', 'box', 'grand', 'hog', 'work', 'wood', 'watch', 'over', 'pony', 'some', 'water', 'pin', 'love', 'beef', 'lace

In [9]:
print("Loading glove embeddings..")
word_index = dict(zip(voc, range(len(voc))))
vocab = torchtext.vocab.GloVe(name='6B',dim=50)
vocab_words = set(vocab.itos)
hits, misses = 0,0
for w in word_index.keys():
    if w in vocab_words:
        hits=hits+1
    else:
        misses = misses+1

print("Word count: ", len(word_index))
print("Hits/misses: ", hits, '/', misses)

print(word_index)




Loading glove embeddings..
Word count:  8550
Hits/misses:  8550 / 0
{'about': 0, 'abouts': 1, 'abundance': 2, 'acetyl': 3, 'acetylcholine': 4, 'ache': 5, 'aches': 6, 'achiever': 7, 'achievers': 8, 'acre': 9, 'acting': 10, 'active': 11, 'activewear': 12, 'activity': 13, 'after': 14, 'afterbirth': 15, 'afterburner': 16, 'afterburners': 17, 'aftercare': 18, 'aftereffect': 19, 'aftereffects': 20, 'afterglow': 21, 'afterimage': 22, 'afterimages': 23, 'afterlife': 24, 'aftermath': 25, 'aftermaths': 26, 'afternoon': 27, 'afternoons': 28, 'aftershock': 29, 'aftershocks': 30, 'aftertaste': 31, 'afterthought': 32, 'afterthoughts': 33, 'afterworld': 34, 'age': 35, 'agent': 36, 'agents': 37, 'ages': 38, 'air': 39, 'airbrake': 40, 'airbrush': 41, 'airburst': 42, 'airbus': 43, 'airbuses': 44, 'aircraft': 45, 'aircraftman': 46, 'aircrew': 47, 'aircrews': 48, 'airdrome': 49, 'airdrop': 50, 'airdrops': 51, 'airfare': 52, 'airfares': 53, 'airfield': 54, 'airfields': 55, 'airflow': 56, 'airfoil': 57, 'ai

In [10]:
# cmp_embeddings = np.array( df.iloc[:int((0.8*(len(compound_data['c1'])))), 104:154], dtype='float32' )
# print(cmp_embeddings[0])
# print(compounds[0])

In [11]:
# x_c1 = [vocab[w] for w in c1]
# x_c2 = [vocab[w] for w in c2]

# print(x_c1[0])
# print(c1[0])
# print(x_c2[0])
# print(c2[0])

In [12]:
glove = ('/content/drive/MyDrive/glove.6B.50d.txt')
print(glove)

/content/drive/MyDrive/glove.6B.50d.txt


In [13]:
# Reading glove text file
with open(glove) as f:
    lines = f.readlines()

glove_dict = OrderedDict()

# converting in key value format
# example ("the" : ["0.22323", "0.0023232", ......])
for i in lines:
  tokens  = i.split(" ")
  key = tokens[0]
  tokens.pop(0)
  value = tokens
  glove_dict[key] = value

print(len(glove_dict))

400000


In [14]:
index = None
if "the" in glove_dict:
    index = list(glove_dict).index("the")
print("Index of the:", index)

Index of the: 0


In [15]:
v = glove_dict['the']

float_glove_dict = OrderedDict()

# converting values from string to float and muliplying to convert from floats to long/int later.
for key, value in glove_dict.items():
  l = []
  for i in value:
    # f = float(i) * 10000
    f = float(i)
    l.append(f)
  float_glove_dict[key] = l

In [16]:
# converting to tensor

count = 0

final = []
input_token = []

list_of_glove_dict = list(float_glove_dict)

for x, y in zip(c1, c2):
  tens = []
  token = []
  if x and y in float_glove_dict:
    # embedding_c1 = float_glove_dict[x]
    # index_1 = list(float_glove_dict).index(x)
    index_1 = list_of_glove_dict.index(x) # This is to get the index of the word for creating token tensors.
    embedding_c1 = float_glove_dict[x] # this is where we are getting embeddings of the word
    # print("index1 = ", index_1)
    token.append(index_1)

    # print(x)
    # print(embedding_c1)
    tens.append(embedding_c1)

    index_2 = list_of_glove_dict.index(y)
    embedding_c2 = float_glove_dict[y]
    # print("index2 = ", index_2)
    token.append(index_2)

    # print(y)
    # print(embedding_c2)
    tens.append(embedding_c2)

    # tens = torch.Tensor(tens)
    final.append(tens)
    input_token.append(token)
    if count == 300:
      break
    count += 1


encode_input = torch.Tensor(final)
# tens = tens.unsqueeze(0)
# print(input_token)
input_token = torch.IntTensor(input_token)

print(input_token)
print(input_token.shape)


print(encode_input)
print(type(encode_input))
print(len(encode_input))
print(encode_input.shape)

tensor([[   298,    430],
        [ 23764,   2632],
        [  5872,   7983],
        [  2642,   7015],
        [  1258,    362],
        [   813,   1145],
        [  1627,   8582],
        [  1176,    817],
        [ 20103,   2555],
        [   430,   4838],
        [  1620,   8518],
        [  2895,    607],
        [  5896,  34855],
        [ 15422,  16013],
        [  2149,   1319],
        [  1620,    534],
        [  6016,    362],
        [  8453,   1146],
        [   156,    226],
        [  8224,   4437],
        [  1707,   3758],
        [  5973,   9051],
        [  3670,   7931],
        [   897,    166],
        [  6504,  16233],
        [   817,   2120],
        [  3918,    484],
        [   539,   7141],
        [   437,   6201],
        [  5973,   9371],
        [  4896,   1370],
        [  1654,   1904],
        [   364,   8606],
        [ 55210,    899],
        [   298,  18366],
        [  1654,   5511],
        [ 23764,   3283],
        [  5171,  18081],
        [  3

In [17]:
# x_c1 = [word_index[w] for w in c1]
# print(c1)
# x_c2 = [word_index[w] for w in c2]
# print(c2)
# print(x_c1)

# xx = [ torch.concat([a,b], axis=0) for a,b in zip(x_c1, x_c2) ]
# print(xx)

In [18]:
count = 0
decode_input = []
final = []
output_token = []

list_of_glove_dict = list(float_glove_dict)


for cmp in compounds:
  tens = []
  token = []
  if cmp in float_glove_dict:

    index_1 = list_of_glove_dict.index(cmp) # This is to get the index of the word for creating token tensors.
    compound = float_glove_dict[cmp] # this is where we are getting embeddings of the word from dict

    # print("index1 = ", index_1)
    token.append(index_1)
    # print(y)
    # print(compound)
    tens.append(compound)
    final.append(tens)
    output_token.append(token)
    if count == 300:
      break
    count += 1

# print(tens[0] * 10)
decode_input = torch.Tensor(final)
# decode_input = decode_input.unsqueeze(0)
# print(output_token)
# tgt_data = output_token.reshape(-1, output_token)
output_token = torch.IntTensor(output_token)

print(output_token)
print(output_token.shape)

print(decode_input)
print(type(decode_input))
print(len(decode_input))
print(decode_input.shape)

tensor([[ 12095],
        [ 44131],
        [ 53542],
        [165374],
        [ 60904],
        [140634],
        [ 67312],
        [ 27562],
        [ 22394],
        [ 12981],
        [365693],
        [ 60783],
        [ 87375],
        [234195],
        [ 47885],
        [135030],
        [ 57247],
        [207694],
        [ 68957],
        [ 91921],
        [100078],
        [ 87160],
        [ 34221],
        [ 11906],
        [148106],
        [137375],
        [ 93185],
        [121079],
        [232393],
        [356866],
        [ 85875],
        [ 28651],
        [ 76171],
        [ 10855],
        [195797],
        [ 72065],
        [110498],
        [133666],
        [ 56282],
        [ 91321],
        [ 72827],
        [379931],
        [150831],
        [280263],
        [173427],
        [184467],
        [103802],
        [ 16171],
        [ 34532],
        [363492],
        [ 60903],
        [ 72096],
        [192377],
        [291784],
        [160508],
        [2

In [19]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)

        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [20]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [21]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # print("Above X ########################")
        # print(x)
        # # x = torch.Tensor(x)
        # # print(x)
        # # x = x.unsqueeze(0)
        # # print(x)
        # print("########################")
        return x + self.pe[:, :x.size(1)]

In [22]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [23]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [24]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        # print(tgt_mask.shape)
        # print(tgt_mask)
        # print(src_mask.shape)
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        # print("src_mask: ", src_mask)
        # print("src_mask shape: ", src_mask.shape)
        # print("tgt_mask: ", tgt_mask)
        # print("tgt_mask shape: ", tgt_mask)
        # print("##############################")
        # print("Src: ", src)
        # print("tgt: ", tgt)
        # print("self.encoder_embedding(src): ", self.encoder_embedding(src))
        # print("self.encoder_embedding(src) shape", self.encoder_embedding(src).shape)
        # print("self.decoder_embedding(tgt): ", self.decoder_embedding(tgt))
        # print("self.decoder_embedding(tgt) shape: ", self.decoder_embedding(tgt).shape)

        enc_embedding = get_enc_embeddings()
        # enc_embedding = self.encoder_embedding(src)

        dec_embedding = get_dec_embeddings()
        # dec_embedding = self.decoder_embedding(tgt)

        # print("printing encoding embeddings = ", enc_embedding)
        # print("printing decoder embeddings = ", dec_embedding)

        src_embedded = self.dropout(self.positional_encoding(enc_embedding))
        tgt_embedded = self.dropout(self.positional_encoding(dec_embedding))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [25]:
# input_ids = encoded_input.input_ids
# input_ids_1 = tens[0]
# input_ids_2 = tens[1]
# print(len(tens))
# print(torch.max(input_ids))
# print(torch.max(input_ids, dim=0))
# input_ids = input_ids.unsqueeze(0)
# print(input_ids_1.shape)
# print(input_ids_2.shape)
# # print(input_ids.unsqueeze(0))
# print(input_ids_1)
# print(type(input_ids_1))
# print(len(input_ids_1[0]))
# attention_mask = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [26]:
# # decoder_input_ids = decoded_input.input_ids

# decoder_input_ids = tens[2]
# print(decoder_input_ids.shape)
# print(decoder_input_ids)
# print(type(decoder_input_ids))
# print(len(decoder_input_ids[0]))
# # decoder_attention_mask = [1, 1, 1, 1, 1, 1, 1]

In [27]:
src_vocab_size = 400000
tgt_vocab_size = 400000
d_model = 50
num_heads = 5
num_layers = 6
d_ff = 2048
max_seq_length = 2
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# src_data = torch.randint(1, src_vocab_size, (2, max_seq_length))  # (batch_size, seq_length)
# src_data = input_token
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (2, max_seq_length))  # (batch_size, seq_length)
# tgt_data = output_token

# print(src_data)
# print(len(src_data))
# print(tgt_data)
# print(tgt_data[:, :-1])

src_data = input_token
print(input_token)
print(input_token[:10])
print(input_token[10:])
traindata = input_token[:100]
val_data = input_token[100:200]
test_data = input_token[200:]
print(src_data)
print(len(src_data))

tgt_data = output_token
y_data = output_token[:100]
y_val_data = output_token[100:200]
y_test_data = output_token[200:]
print(tgt_data)

print(tgt_data)
# print(tgt_data[:, :-1])

print(src_data.shape)
print(tgt_data.shape)


tensor([[   298,    430],
        [ 23764,   2632],
        [  5872,   7983],
        [  2642,   7015],
        [  1258,    362],
        [   813,   1145],
        [  1627,   8582],
        [  1176,    817],
        [ 20103,   2555],
        [   430,   4838],
        [  1620,   8518],
        [  2895,    607],
        [  5896,  34855],
        [ 15422,  16013],
        [  2149,   1319],
        [  1620,    534],
        [  6016,    362],
        [  8453,   1146],
        [   156,    226],
        [  8224,   4437],
        [  1707,   3758],
        [  5973,   9051],
        [  3670,   7931],
        [   897,    166],
        [  6504,  16233],
        [   817,   2120],
        [  3918,    484],
        [   539,   7141],
        [   437,   6201],
        [  5973,   9371],
        [  4896,   1370],
        [  1654,   1904],
        [   364,   8606],
        [ 55210,    899],
        [   298,  18366],
        [  1654,   5511],
        [ 23764,   3283],
        [  5171,  18081],
        [  3

In [28]:
# print(encode_input)
print(encode_input[:10])
print(decode_input[:10])
def get_enc_embeddings():
  # print(encode_input)
  return encode_input[:100]

def get_dec_embeddings():
  return decode_input[:100]

tensor([[[-6.8652e-01,  8.0125e-01, -6.1240e-01, -1.5120e-01,  9.9700e-01,
           8.1749e-01, -1.2089e+00, -1.0221e+00, -9.4054e-01, -1.2689e+00,
          -6.8195e-01, -1.2212e-01,  6.1220e-01,  4.5025e-02, -1.0808e-01,
           1.2482e-01, -1.7278e-01, -4.5383e-01, -3.7728e-01, -1.4081e+00,
          -4.2975e-01,  3.1746e-01,  6.1391e-01, -7.6399e-01, -7.5504e-01,
          -1.5605e+00, -1.4924e-01,  1.2023e+00,  3.6776e-01, -4.9137e-01,
           2.6151e+00, -3.7237e-01, -8.0641e-01, -5.6119e-01,  2.8241e-02,
          -2.0089e-01, -3.6429e-01, -5.1476e-01, -2.3074e-01, -7.2258e-01,
          -8.6035e-02,  4.7341e-01, -3.5288e-02, -4.8920e-02,  8.5295e-01,
          -2.5883e-01, -4.8159e-01, -7.9552e-01, -1.2247e-01, -9.8168e-02],
         [ 5.3507e-01,  5.7610e-01, -5.4351e-02, -2.0800e-01, -7.8820e-01,
          -1.7592e-01, -2.1255e-01, -1.4388e-01,  1.0344e+00, -7.9253e-02,
           2.7696e-01,  3.7951e-01,  1.2139e+00, -3.4032e-01, -1.8118e-01,
           7.2968e-01,  

In [29]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(traindata, y_data)
    print("output = ",output)
    new_tgt = y_data.long()
    # print(new_tgt)

    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), new_tgt.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

output =  tensor([[[ 0.3982,  0.4596,  0.8492,  ...,  0.2445,  1.1250, -0.1564]],

        [[ 0.5593, -0.0555, -0.4051,  ...,  0.6039,  0.6110, -0.7405]],

        [[ 1.0149, -0.1291,  1.1239,  ...,  1.0883,  0.0449,  0.1146]],

        ...,

        [[ 0.2106,  0.1510,  0.9199,  ...,  0.7815,  0.0758,  0.0763]],

        [[ 0.6414, -0.5120,  0.8292,  ...,  0.5884,  0.6183,  0.8072]],

        [[ 0.5088, -0.0157,  1.2420,  ...,  0.5911,  0.8505,  0.3063]]],
       grad_fn=<ViewBackward0>)
Epoch: 1, Loss: 13.124837875366211
output =  tensor([[[ 0.1949,  0.8597,  0.8436,  ...,  0.1101,  1.2301,  0.0072]],

        [[ 0.3239,  0.2902,  0.0506,  ...,  1.2895,  0.2483,  0.0290]],

        [[ 0.8408, -0.0529,  0.9706,  ...,  0.6037, -0.0288,  0.2908]],

        ...,

        [[ 0.5536,  0.3169,  1.0089,  ...,  0.2427,  0.2733,  0.0467]],

        [[ 0.4933, -0.7637,  1.2655,  ...,  0.4439,  0.1361,  0.1340]],

        [[ 0.0337, -0.2479,  1.2605,  ...,  0.2166,  0.7176,  0.5133]]],
       gr

In [30]:
# output = transformer.predict(test_data)

In [32]:
print(encode_input)
print(encode_input[10:])
print(decode_input[10:])

print(x)

def get_enc_embeddings():
  # print(encode_input)
  return encode_input[100:200]

def get_dec_embeddings():
  return decode_input[100:200]

tensor([[[-0.6865,  0.8012, -0.6124,  ..., -0.7955, -0.1225, -0.0982],
         [ 0.5351,  0.5761, -0.0544,  ..., -0.7865,  0.6156, -0.9548]],

        [[-0.5597, -0.4002, -0.5693,  ...,  0.2874, -0.0127,  0.0759],
         [ 0.7868,  0.0794, -0.7660,  ..., -0.2536,  0.2642,  0.1587]],

        [[-0.3744, -0.1196, -0.8761,  ..., -0.1014, -0.5113, -0.1863],
         [-1.0387, -0.3306, -0.3527,  ...,  0.1166,  0.9017, -0.6902]],

        ...,

        [[-0.6676,  0.1041, -0.1908,  ..., -0.1842,  0.4954, -0.4509],
         [-0.7839,  0.2743,  0.2905,  ...,  0.0397, -0.4634,  0.5552]],

        [[-0.2362,  0.2753,  0.5688,  ..., -0.5087, -0.5688, -0.9219],
         [ 0.2559, -0.3145, -1.0079,  ..., -0.8059,  0.1649,  0.2979]],

        [[-0.0185,  0.4392,  0.6476,  ...,  0.6728, -0.3650,  0.7077],
         [-0.2036,  0.1008, -0.0406,  ..., -1.5619, -0.0118, -0.0127]]])
tensor([[[-4.6177e-02,  1.9105e-01, -8.8775e-03,  ...,  2.0321e-01,
           3.1264e-01,  9.3996e-01],
         [ 4.7061

In [33]:
transformer.eval()

# Generate random sample validation data
# val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_data, y_val_data)

    print(val_output)

    new_test_tgt = y_val_data.long()

    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), new_test_tgt.contiguous().view(-1))

    print(f"Validation Loss: {val_loss.item()}")

tensor([[[-0.6194,  0.6563, -0.5752,  ...,  0.5819,  0.1009,  0.2724]],

        [[-0.7854,  0.2221, -0.1235,  ..., -0.8017,  1.3770,  0.2576]],

        [[-0.5483, -0.9600,  1.3553,  ..., -0.7305, -0.1537, -0.6471]],

        ...,

        [[-1.1552, -0.6683,  0.6610,  ..., -1.5100,  0.3329,  0.4562]],

        [[-0.2201,  0.5338, -0.1311,  ...,  1.1591,  0.3166,  0.2541]],

        [[-1.0169,  0.4425, -0.3682,  ..., -0.3755,  1.0268,  0.6112]]])
Validation Loss: 13.051636695861816


In [34]:
# print(encode_input)
# print(encode_input[10:])
# print(decode_input[10:])

x = np.zeros((101, 1, 50))

x = torch.Tensor(x)

# print(x)

def get_enc_embeddings():
  # print(encode_input)
  return encode_input[200:]

def get_dec_embeddings():
  return x

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])


In [ ]:
# prompt: write code to make a vector of zeros

x = np.zeros((101, 1))

x = torch.Tensor(x)

# print(x)

transformer.eval()

# Generate random sample validation data
# val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    test_output = transformer(test_data, x)

    print(test_output)

    new_test_tgt = y_test_data.long()

    test_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), new_test_tgt.contiguous().view(-1))

    print(f"test Loss: {test_loss.item()}")

In [ ]:
def find_closest_embeddings(vocab, embedding):
    return sorted(vocab.itos[:1000], key=lambda word: spatial.distance.euclidean(vocab[word], embedding))
